# Data Management Using `pandas`, Part 2

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

## Problem 0
Import the following libraries:

In [ ]:
import numpy as np
import pandas as pd

## Problem 1
In the first part of this lab, the goal is to merge data from the United Nations World Health Organization (https://www.who.int/who-un/en/) with data from the Varieties of Democracy Project (https://www.v-dem.net/en/). The UN-WHO studies health outcomes in a cross-national context, and V-Dem studies the quality of democracy as it changes across countries and over time. We would want to merge these two datasets together if we wanted to study whether democratic quality can predict health outcomes.

The UN data contains cross-national time series data from the United Nations and World Health Organization, and includes three features:

* The number of physicians per 1000 people
* The percent of the population that is malnourished
* Health expenditure per capita

The VDem data comes from the Varieties of Democracy project, which aims to measure the quality of democracy and the amount of corruption in different countries over time (https://www.v-dem.net/en/data/data-version-8/). This data file contains indices regarding a country’s democractic quality, level of civil liberites, and corruption. It also contains a binary indicator that separates countries into democratic and nondemocratic states, and it includes a categorizaton of the corruption scale.

The URLs for the two datasets are:

In [ ]:
undata_url = "https://github.com/jkropko/DS-6001/raw/master/localdata/UNdata.csv"
VDem_url = "https://github.com/jkropko/DS-6001/raw/master/localdata/vdem.csv"

### Part a
Load both CSV files. Make sure to check whether there are rows that should not be included in the dataframe, and whether there are missing codes that should be replaced with `NaN`. Fix these problems at the data loading stage, if you can. (Don't worry about column names or category labels yet.) Also, the UN data covers the years 1960-2014, and the VDem data covers the years 1960-2015. To make the timeframe match up, delete rows in the VDem data from 2015. (1 point)

### Part b
The UN data contain certain rows that refer to groups of countries instead of to individual countries. Here’s a list of these non-countries:

In [ ]:
noncountries = ['Arab World',  'Caribbean small states',  'Central Europe and the Baltics', 
    'Early-demographic dividend',  'East Asia & Pacific', 'East Asia & Pacific (excluding high income)', 
    'East Asia & Pacific (IDA & IBRD countries)', 'Euro area', 'Europe & Central Asia', 
    'Europe & Central Asia (excluding high income)', 'Europe & Central Asia (IDA & IBRD countries)', 'European Union', 
    'Fragile and conflict affected situations', 'Heavily indebted poor countries (HIPC)', 
    'High income', 'Late-demographic dividend', 'Latin America & Caribbean', 
    'Latin America & Caribbean (excluding high income)', 
    'Latin America & the Caribbean (IDA & IBRD countries)', 'Least developed countries: UN classification', 
    'Low & middle income', 'Low income', 'Lower middle income', 
    'Middle East & North Africa', 'Middle East & North Africa (excluding high income)',
    'Middle East & North Africa (IDA & IBRD countries)', 
    'Middle income', 'North America', 'OECD members', 
    'Other small states', 'Pacific island small states', 'Post-demographic dividend', 
    'Pre-demographic dividend', 'Small states', 'South Asia', 
    'South Asia (IDA & IBRD)', 'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)', 
    'Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income', 'World']

We can use `.query()` to remove the non-countries from the data, but in this case there are complications due to the space in the name of the column `Country Name` and the use of an external list. So here let's use an alternative method:

First, apply the `.isin(noncountries)` method to the `Country Name` column of the UN data to create a series of values that are `True` if the `Country Name` on a row is one of the non-countries, and `False` otherwise. Second, use the `~` operator to negate the logical values: turn `True` to `False` and vice versa. Finally, pass this logical series to the `.loc[]` attribute of the dataframe to drop the rows that refer to these noncountries from the UN data. (1 point)

(If you wanted to use `.query()`, you would first need to rename `Country Name` to remove the space, then you can use an `@` in front of `noncountries` to refer to the external list. But for this problem follow the instructions listed above.)

### Part c
Reshape the UN data to move the years from the columns to the rows. (Once the years are in the rows, they will have values such as "1960 [YR1960]".) (2 points)

### Part d
Rename the `variable` column to `year`. Then use string methods to remove the ends such as "[YR1960]" from the values of the new `year` column and convert the column to an integer data type.

Also, for whatever reason, real world data often contains multiple variables that are just different representations of the same information. In this case, the `Series Name` and `Series Code` variables tell us exactly the same thing, and the `Country Name` and `Country Code` variables tell us exactly the same thing. Unless I have a very good reason to keep both, I generally prefer to drop variables that are redundant and coded in a less helpful way. So drop `Series Code` and `Country Code`. (2 points)

### Part e
Reshape the data to move the values of `Series Name` to separate columns. Make sure all of the columns exist in the dataframe after reshaping and are not stored in a row index or multi-index. Then rename the columns so that all of the columns have concise and descriptive names. (2 points)

### Part f
Next we are going to join the cleaned UN data with the VDem data. In a perfect world, both datasets would include a shared numeric country ID field that we can use to match countries in one dataset to countries in the other. Unfortunately the UN data identifies the countries only by name. Worse still, while there is a big overlap the two datasets cover different sets of countries.

First decide whether this merge is a one-to-one, one-to-many, many-to-one, or many-to-many merge and describe your rationale in words.

Then perform a test merge that checks whether your expectation that the merge is one-to-one, one-to-many, many-to-one, or many-to-many is confirmed, and reports whether each row is matched, appears only in the UN data, or appears only in the VDem data. Use the `.unique()` or `.value_counts()` method to display the names of the countries that are not matched. (2 points)

### Part g
There are many unmatched rows in this merge. There are three reasons why rows failed to match:
* Differences in geographical coverage: for example, the VDem data includes Taiwan, but the UN data does not
* Differences in time coverage: for example, the UN data includes records for France every year from 1970 through 2014, and VDem includes rows for France from 1960 to 2012, leaving 12 rows for France without matching years
* Differences in spelling: for example, South Korea is called "Korea, Rep." in the UN data and "Korea_South" in the VDem data.

We can't do anything about differences in geographic or temporal coverage. But we can recode some country names to account for differences in spelling and to match more rows that should match. Here is a list of differently spelled countries:

* "Burma_Myanmar" in VDem is "Myanmar" in the UN data
* "Cape Verde" in VDem is "Cabo Verde" in the UN data
* "Congo_Democratic Republic of" in VDem is "Congo, Dem. Rep." in the UN data
* "Congo_Republic of the" in VDem is "Congo, Rep." in the UN data
* "East Timor" in VDem is "Timor-Leste" in the UN data
* "Egypt" in VDem is "Egypt, Arab Rep." in the UN data
* "Gambia" in VDem is "Gambia, The" in the UN data
* "Iran" in VDem is "Iran, Islamic Rep." in the UN data
* "Ivory Coast" in VDem is "Cote d’Ivoire" in the UN data
* "Korea_North" in VDem is "Korea, Dem. People’s Rep." in the UN data
* "Korea_South" in VDem is "Korea, Rep." in the UN data
* "Kyrgyzstan" in VDem is "Kyrgyz Republic" in the UN data
* "Laos" in VDem is "Lao PDR" in the UN data
* "Macedonia" in VDem is "Macedonia, FYR" in the UN data
* "Palestine_West_Bank" in VDem is "West Bank and Gaza" in the UN Data (there is also "Palestine_Gaza" in VDem, but since the UN combines data for the West Bank and Gaza, let's just use "Palestine_West_Bank" for this assignment)
* "Russia" in VDem is "Russian Federation" in the UN data
* "Slovakia" in VDem is "Slovak Republic" in the UN data
* "Syria" in VDem is "Syrian Arab Republic" in the UN data
* "Venezuela" in VDem is "Venezuela, RB" in the UN data
* "Vietnam_Democratic Republic of" in VDem is "Vietnam" in the UN data
* "Yemen" in VDem is "Yemen, Rep." in the UN data

Recode the country names listed above in one of the two dataframes to match the names in the other dataframe. Then perform an inner join of the two dataframes. Some rows will be dropped because of differences in coverage, but no rows will be dropped because of differences in spelling. (2 points)

## Problem 2
[Kickstarter](https://www.kickstarter.com/) is a website in which people can pledge financial support for creative projects. Patrons are only charged if a project raises enough money to meet a pre-specified goal, and projects can offer items as "rewards" for patrons who contribute at particular levels. One interesting aspect of Kickstarter is the ability to [search projects by "ending soon"](https://www.kickstarter.com/discover/advanced?sort=end_date). If you have a few dollars to spare and want to feel like a hero, you can swoop in at the last minute to contribute enough for a project to meet its goal.

Cathie So created a project on Kaggle in which she [scraped Kickstarter](https://www.kaggle.com/socathie/kickstarter-project-statistics/data?select=live.csv) and collected data on 4000 live projects (projects that were currently collecting pledges from patrons) as of October 10, 2016, at 5pm Pacific time. The data are here:

In [ ]:
kickstarter = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/live.csv")
kickstarter

,Unnamed: 0,amt.pledged,blurb,by,country,currency,end.time,location,percentage.funded,state,title,type,url
0,0,15823.0,"\n'Catalysts, Explorers & Secret Keepers: Wome...",Museum of Science Fiction,US,usd,2016-11-01T23:59:00-04:00,"Washington, DC",186,DC,"Catalysts, Explorers & Secret Keepers: Women o...",Town,/projects/1608905146/catalysts-explorers-and-s...
1,1,6859.0,\nA unique handmade picture book for kids & ar...,"Tyrone Wells & Broken Eagle, LLC",US,usd,2016-11-25T01:13:33-05:00,"Portland, OR",8,OR,The Whatamagump (a hand-crafted story picture ...,Town,/projects/thewhatamagump/the-whatamagump-a-han...
2,2,17906.0,\nA horror comedy about a repairman who was in...,Tessa Stone,US,usd,2016-11-23T23:00:00-05:00,"Los Angeles, CA",102,CA,Not Drunk Enough Volume 1!,Town,/projects/1890925998/not-drunk-enough-volume-1...
3,3,67081.0,\nThe Johnny Wander autobio omnibus you've all...,Johnny Wander,US,usd,2016-11-01T23:50:00-04:00,"Brooklyn, NY",191,NY,Our Cats Are More Famous Than Us: A Johnny Wan...,County,/projects/746734715/our-cats-are-more-famous-t...
4,4,32772.0,\nThe vision for this project is the establish...,Beau's All Natural Brewing Company,RW,cad,2016-11-18T23:05:48-05:00,"Kigali, Rwanda",34,Kigali Province,The Rwanda Craft Brewery Project,Town,/projects/beaus/the-rwanda-craft-brewery-proje...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3995,4403.0,\nEARTH IS BUT ONE FRUIT ON THE TREE OF LIFE. ...,Lewis Brown,US,usd,2016-11-20T01:10:00-05:00,"Denver, CO",88,CO,BROWN HORNET OMNIVERSE,Town,/projects/brownhornetomni/brown-hornet-omniver...
3996,3996,1304.0,\nImagine designing an item with an easy-to-us...,Your Expressions,US,usd,2016-11-15T16:00:00-05:00,"San Francisco, CA",5,CA,3D Pixie - App to Design Personalized Jewelry,Town,/projects/yourexpressions/3d-pixie-app-to-crea...
3997,3997,1.0,\nUnique themed London venue and hostel for 9g...,Martin Wojtala,GB,gbp,2016-10-30T09:36:06-04:00,"London, UK",0,England,9HUB - London,Town,/projects/1132099243/9hub-london?ref=discovery
3998,3998,10.0,\nAll in One Phone Case\n,All in One Phone Case,US,usd,2016-11-17T12:11:26-05:00,"Tallahassee, FL",0,FL,All in One Phone Case,Town,/projects/203104559/all-in-one-phone-case?ref=...


### Part a
Notice that the `end.time` column, the date and time at which the project stops accepting pledges, is formatted as follows:
```
2016-11-01T23:59:00-04:00
```
This formatting is "YYYY-MM-DDThh:mm:ss-TZD": four digits for the year, a dash, two digits for the month, another dash, and two digits for the day; the "T" separates the dates from the time; two digits for the hour, minute and second, separated by colons; and the time zone expressed as hours difference from Greenwich mean time (also called UTC), and -04:00 is four hours earlier than UTC, for example.

But `end.time` is also currently read as a string, with `object` data type:

In [ ]:
kickstarter.dtypes

Unnamed: 0             int64
amt.pledged          float64
blurb                 object
by                    object
country               object
currency              object
end.time              object
location              object
percentage.funded      int64
state                 object
title                 object
type                  object
url                   object
dtype: object

Convert `end.time` to a timestamp, and extract the month, day, year, hour, minute, and second of the end time. To allow the `pd.to_datetime()` function to read timezones, use the `utc=True` argument. (2 points)

### Part b
Create a dataframe with one row for every ending day in the `kickstarter` data that reports the average amount pledged (`amt.pledged`) on each day. Sort the rows in descending order by average amount pledged, and display the five days with the highest averages. (2 points)

### Part c
Display the text of the longest `blurb` in the data. (2 points)

### Part d
How many blurbs for projects with end dates between November 15, 2016 and December 7, 2016 contain the phrase "science fiction"? [Hint: Don't forget to make this search case-insensitive and to sort the `kickstarter` dataframe by `end.time` before setting `end.time` as the index.] (2 points)